# Illustration of how synaptic input becomes dipolar with distance

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
from brainsignals.plotting_convention import mark_subplots, cmap_v_e
from brainsignals.neural_simulations import return_hay_cell

np.random.seed(12345)


In [ ]:
# Create a grid of measurement locations zoomed in, in (um)
grid_x, grid_z = np.mgrid[-650:651:20, -1000:1200:20]
grid_y = np.ones(grid_x.shape) * 0

# Create a grid of measurement locations zoomed out, in (um)
grid_X, grid_Z = np.mgrid[-6000:6001:200, -6500:7200:200]
grid_Y = np.ones(grid_X.shape) * 0

sigma = 0.3
# Define electrode parameters
grid_elec_params_in = {
    'sigma': sigma,      # extracellular conductivity
    'x': grid_x.flatten(),  # electrode positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten(),
}

# Define electrode parameters
grid_elec_params_out = {
    'sigma': sigma,      # extracellular conductivity
    'x': grid_X.flatten(),  # electrode positions
    'y': grid_Y.flatten(),
    'z': grid_Z.flatten(),
}

ax_in_lfp_dict = dict(aspect=1, frameon=False, xticks=[], yticks=[],
                   ylim=[np.min(grid_z), np.max(grid_z)],
                   xlim=[np.min(grid_x), np.max(grid_x)])

ax_out_lfp_dict = dict(aspect=1, frameon=False, xticks=[], yticks=[],
                   ylim=[np.min(grid_Z), np.max(grid_Z)],
                   xlim=[np.min(grid_X), np.max(grid_X)])


In [ ]:
def insert_synaptic_input(idx, cell):

    synapse_parameters = {'e': 0., # reversal potential
                          'weight': 0.002, # synapse weight
                          'record_current': True, # record synapse current
                          'syntype': 'Exp2Syn',
                          'tau1': 1, #Time constant, rise
                          'tau2': 3, #Time constant, decay
                          }
    synapse_parameters['idx'] = idx
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([1.]))
    return synapse, cell


def plot_grid_LFP(cell, grid_elec_params, grid_x, grid_z, ax, synapses, ms=5,
                  mec='k', scale_max=None):

    grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
    M_elec_ps = grid_electrode.get_transformation_matrix()
    lfp_ = M_elec_ps @ cell.imem * 1e6

    max_amp_elec_idx = np.argmax(np.max(np.abs(lfp_), axis=1))
    max_amp_t_idx = np.argmax(np.abs(lfp_[max_amp_elec_idx, :]))

    max_amp_LFP = np.max(np.abs(lfp_))
    if not max_amp_LFP == np.abs(lfp_[max_amp_elec_idx, max_amp_t_idx]):
        raise RuntimeError("Wrong with chosen max value")

    LFP = lfp_[:, max_amp_t_idx].reshape(grid_x.shape)

    num = 11
    levels = np.logspace(-2.2, 0, num=num)

    scale_max = np.max(np.abs(LFP)) if scale_max is None else scale_max

    levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

    colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                       for i in range(len(levels_norm) - 1)]
    colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)
    
    ax.plot(cell.x.T, cell.z.T, lw=1, c='gray', zorder=1000)

    [ax.plot(cell.x[syn.idx].mean(), cell.z[syn.idx].mean(),
             marker='o', c='b', ms=ms, mec=mec, zorder=1000)
     for syn in synapses]

    ep_intervals = ax.contourf(grid_x, grid_z, LFP,
                                   zorder=2, colors=colors_from_map,
                                   levels=levels_norm, extend='both')

    ax.contour(grid_x, grid_z, LFP, colors='k', linewidths=(1), zorder=2,
                   levels=levels_norm)
    return ep_intervals


In [ ]:
chosen_basal_idxs = np.array([621, 800, 95])

plt.close("all")
fig = plt.figure(figsize=[6, 5])
fig.subplots_adjust(bottom=0.10, top=0.98, right=0.99,
                    left=0.0, wspace=-0.1, hspace=0.)
num_cols = 3
tstop = 10
dt = 2**-4

scalemax_zoomed_in = 1000
scalemax_zoomed_out = 15

# Plot single-synapse dipoles
axes_to_mark = []
for i, syn_idx in enumerate(chosen_basal_idxs):
    synapses = []
    cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
    syn, cell = insert_synaptic_input(syn_idx, cell)
    synapses.append(syn)
    cell.simulate(rec_imem=True)

    ax_in = fig.add_subplot(2, num_cols, i + 1, **ax_in_lfp_dict)
    axes_to_mark.append(ax_in)

    ep_intervals_in = plot_grid_LFP(cell, grid_elec_params_in, grid_x, grid_z,
                                 ax_in, synapses, ms = 3, mec = 'b',
                                 scale_max=scalemax_zoomed_in)

    ax_out = fig.add_subplot(2, num_cols, i + 4, **ax_out_lfp_dict)
    axes_to_mark.append(ax_out)

    ep_intervals_out = plot_grid_LFP(cell, grid_elec_params_out, grid_X, grid_Z,
                                 ax_out, synapses, ms = 3, mec = 'b',
                                 scale_max=scalemax_zoomed_out)

    if i == 1:
        ax_in.plot([-1050, -1050], [-400, 100], c='k', lw=1, clip_on=False)
        ax_in.text(-1000, -250, "0.5 mm", ha='left', va="center",
                   clip_on=False, zorder=10000)
        ax_out.plot([-6900, -6900], [-5000, 0], c='k', lw=1, clip_on=False)
        ax_out.text(-6700, -2500, "5 mm", ha='left', va="center",
                   clip_on=False, zorder=10000)

    del cell
    del syn

axes_to_mark = [axes_to_mark[0], axes_to_mark[2],
                axes_to_mark[4], axes_to_mark[1],
                axes_to_mark[3], axes_to_mark[5]]

mark_subplots(axes_to_mark, xpos=0.13, ypos=0.96)

cax = fig.add_axes([0.05, 0.6, 0.9, 0.01], frameon=False)
cbar = fig.colorbar(ep_intervals_in, cax=cax, orientation='horizontal')
cbar.set_label('nV', labelpad=0)
cbar.set_ticks(np.array([-1000, -100, -10, 10, 100, 1000]))

cax = fig.add_axes([0.05, 0.08, 0.9, 0.01], frameon=False)
cbar = fig.colorbar(ep_intervals_out, cax=cax, orientation='horizontal')
cbar.set_label('nV', labelpad=0)
cbar.set_ticks([-10, -1, -0.1, 0.1, 1, 10])
cbar.set_ticklabels(["-10", "-1", "", "", "1", "10"])

fig.savefig("fig_single_input_zoomed_out.pdf")